In [45]:
import pandas as pd
import os
import protfasta
import glob
from Bio.Seq import Seq
import pickle

First, downloading the transcripts

In [3]:
! mkdir /Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca

mkdir: /Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca: File exists


In [4]:
all_TFs = pd.read_csv("../soto_analysis/outputs/all_TFs_table_proteins.txt", sep = "\t", index_col = 0)
all_TFs

,1,2,uniprotID,ENSG,ENST,DBD_coords,AD_coords,RD_coords,Bif_coords,length
Unnamed: 0,,,,,,,,,,
0,NaN,NaN,A6NJG6,NaN,ENST00000334384,79-135,142-315,NaN,NaN,1
1,NaN,NaN,A8MTJ6,NaN,ENST00000428390,145-234,369-420,NaN,NaN,1
2,NaN,NaN,A8MYZ6,NaN,ENST00000641094,89-178,382-491,NaN,NaN,1
3,NaN,NaN,A8MZ59,NaN,ENST00000640845,31-65,92-181,NaN,NaN,1
4,NaN,NaN,O00321,NaN,ENST00000403402,242-321,1-162,22-121,NaN,1
...,...,...,...,...,...,...,...,...,...,...
441,NaN,NaN,Q9Y5R6,NaN,ENST00000382276,72-118,172-251,182-261,NaN,1
442,NaN,NaN,Q9Y5W3,NaN,ENST00000248071,"272-296,302-326,332-354",1-89,NaN,NaN,1
443,NaN,NaN,Q9Y692,NaN,ENST00000294409,89-165,422-573,"62-151,212-301",NaN,1


In [5]:
SFARI_TFs = pd.read_csv("../data/SFARI_TFs_with_ENST_corrected.csv")
SFARI_TFs

,Unnamed: 0,uniprotID,ENST
0,0,Q9H2P0,ENST00000349014.8
1,1,Q5TGY3,ENST00000247087.10
2,2,Q96QS3,ENST00000379044.5
3,3,Q96JM3,ENST00000361283.4
4,4,Q96RK0,ENST00000575354.6
...,...,...,...
117,117,Q8NEK5,ENST00000366197.9
118,118,Q9BR84,ENST00000393883.6
119,119,Q68DY1,ENST00000601440.6
120,120,Q9Y462,ENST00000276123.7


In [6]:
SFARI_TFs_Table = all_TFs[all_TFs["uniprotID"].isin(SFARI_TFs["uniprotID"])]
SFARI_TFs_Table

,1,2,uniprotID,ENSG,ENST,DBD_coords,AD_coords,RD_coords,Bif_coords,length
Unnamed: 0,,,,,,,,,,
12,NaN,NaN,O14770,NaN,ENST00000561208,277-334,340-477,150-193,NaN,1
41,NaN,NaN,O75840,NaN,ENST00000309446,"219-243,249-273,279-301",2-101,2-111,NaN,1
43,NaN,NaN,O94983,NaN,ENST00000348066,37-149,"285-468,472-581",182-261,NaN,1
45,NaN,NaN,O95096,NaN,ENST00000377142,129-185,220-273,8-18,NaN,1
74,NaN,NaN,P10275,NaN,ENST00000374690,558-627,"1-494,652-910","322-461,482-591",NaN,1
76,NaN,NaN,P10827,NaN,ENST00000264637,51-122,1-52,NaN,NaN,1
80,NaN,NaN,P11308,NaN,ENST00000288319,312-391,"433-479,118-261",382-461,NaN,1
81,NaN,NaN,P11473,NaN,ENST00000395324,22-91,"415-427,195-238",222-301,NaN,1
98,NaN,NaN,P15884,NaN,ENST00000356073,565-618,"1-100,340-400","2-91,160-179,511-540",NaN,1


In [7]:
# Use uniprot to get gene names 
SFARI_TFs_Table[["uniprotID"]].drop_duplicates().to_csv("../data/SFARI_TF_uniprotIDs.csv", index = None, header = None)

In [8]:
gene_names = pd.read_csv("../data/SFARI_TF_gene_names.tsv", sep = "\t")
gene_names["Gene Names"] = gene_names["Gene Names"].str.split(" ").str[0]
gene_names = gene_names[["From", "Gene Names"]]
gene_names = gene_names.rename(columns = {"From" : "uniprotID", "Gene Names" : "gene"})
gene_names

,uniprotID,gene
0,O14770,MEIS2
1,O75840,KLF7
2,O94983,CAMTA2
3,O95096,NKX2-2
4,P10275,AR
5,P10827,THRA
6,P11308,ERG
7,P11473,VDR
8,P15884,TCF4
9,P19532,TFE3


In [9]:
input_tbl = pd.merge(SFARI_TFs_Table[["uniprotID", "ENST"]], gene_names, on = "uniprotID", how = "left")
input_tbl

,uniprotID,ENST,gene
0,O14770,ENST00000561208,MEIS2
1,O75840,ENST00000309446,KLF7
2,O94983,ENST00000348066,CAMTA2
3,O95096,ENST00000377142,NKX2-2
4,P10275,ENST00000374690,AR
5,P10827,ENST00000264637,THRA
6,P11308,ENST00000288319,ERG
7,P11473,ENST00000395324,VDR
8,P15884,ENST00000356073,TCF4
9,P19532,ENST00000315869,TFE3


In [12]:
# Clicked on links to download, then moved

for ENST, gene in zip(input_tbl["ENST"], input_tbl["gene"]):
    # print(ENST,gene)
    # command = "wget --no-check-certificate https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/" + ENST + "." + gene + ".fasta.gz -P /Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca"
    # os.system(command)

    print("https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/" + ENST + "." + gene + ".fasta.gz")

https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000561208.MEIS2.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000309446.KLF7.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000348066.CAMTA2.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000377142.NKX2-2.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000374690.AR.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000264637.THRA.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000288319.ERG.fasta.gz
https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/ENST00000395324.VDR.fasta.gz
https://genome.senckenberg.de/download/TOGA/hum

In [62]:
# Focusing on smallest file, YY1 to test
files = glob.glob("../data/zoonomia_toga_mca/ENST00000239243.MSX2.fasta.gz")
files

['../data/zoonomia_toga_mca/ENST00000239243.MSX2.fasta.gz']

In [63]:
#Dna transcripts for ref seq
human_dna_transcripts = pickle.load(open('../soto_analysis/raw_files/dna_transcripts.dat', 'rb'))

In [65]:
for file in files:
    #Uncompress - run once
    os.system("gunzip " + file)

    # Read in as dataframe
    df = protfasta.read_fasta(file[:-3], invalid_sequence_action = 'convert')
    df = pd.DataFrame({"id" : df.keys(), "nt_seq" : df.values()})
    df["prot_seq"] = [str(Seq(_).translate()) for _ in df["nt_seq"]]


    ENST = file.split("/")[-1].split(".")[0]
    gene =  file.split("/")[-1].split(".")[1]
    
    prot_dict = dict(zip(df["id"], df["prot_seq"]))
    prot_dict["human"] = str(Seq(human_dna_transcripts[ENST]).translate())
    
    protfasta.write_fasta(prot_dict, "../data/zoonomia_toga_mca/prot_fastas/" + gene)

In [56]:
# Everything seems very conserved